In [2]:
import pandas as pd
from utility import *
from scipy.stats import sem


In [3]:
# several metrics to consider
# mean length bias
# power
# false positive rate for various lengths(# of FP per pair of samples)

cov = 'cov1over4'
ibd_ins = [0.1, 1, 10]
ibd_outs = [1, 10, 50]
ibd_jumps = [100, 250, 500, 1000]
gaps = [0.005, 0.0075, 0.01]
post_cutoffs = [0.99, 0.995, 0.999]

for cov in ['cov1', 'cov1over2', 'cov1over4']:
    metrics = []
    for i, ibd_in in enumerate(ibd_ins):
        for j, ibd_out in enumerate(ibd_outs):
            for k, ibd_jump in enumerate(ibd_jumps):
                for m, gap in enumerate(gaps):
                    for n, cutoff in enumerate(post_cutoffs):
                        ret, power = lengthCalledOneBatch(f'./grid/{cov}/ch3_12cm/ibd_in{i}_ibd_out{j}_ibd_jump{k}_gap{m}_cutoff_post{n}/ch3.tsv', \
                                './simGeno/ch3_12cm/ibd_info.csv')
                        df_fp = pd.read_csv(f'./grid/{cov}/ch3_0cm/ibd_in{i}_ibd_out{j}_ibd_jump{k}_gap{m}_cutoff_post{n}/ch3.tsv', sep='\t')
                        df_rate6 = len(df_fp[df_fp['lengthM']>=0.06])/500
                        df_rate8 = len(df_fp[df_fp['lengthM']>=0.08])/500
                        df_rate10 = len(df_fp[df_fp['lengthM']>=0.1])/500
                        df_rate12 = len(df_fp[df_fp['lengthM']>=0.12])/500
                        metrics.append([ibd_in, ibd_out, ibd_jump, gap, cutoff, df_rate6, df_rate8, df_rate10, df_rate12, power, np.mean(ret), sem(ret)])
    
    df = pd.DataFrame(metrics, columns=['ibd_in', 'ibd_out', 'ibd_jump', 'max_gap_merge', 'post_cutoff', \
            'FP(>=6cM)', 'FP(>=8cM)', 'FP(>=10cM)', 'FP(>=12cM)', 'power', 'mean inferred IBD lengths', 'SE of inferred IBD lengths'])
    df.to_csv(f'./grid/grid.wgs.{cov}.csv')